Data 

*   https://analyticsindiamag.com/10-open-source-datasets-one-must-know-to-build-recommender-systems/
*   Book rating

In [1]:
!pip install pyspark
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import warnings
import time
from scipy.spatial.distance import cosine
warnings.filterwarnings('ignore')
sc = SparkSession \
       .builder \
       .config("spark.driver.memory", "15g") \
       .appName("Data") \
       .getOrCreate()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
raw = pd.read_csv('/content/drive/MyDrive/Clouds/Cources/BigData/midterm/BX-Book-Ratings.csv', encoding = 'unicode_escape',error_bad_lines=False,sep=";")
raw_books = pd.read_csv('/content/drive/MyDrive/Clouds/Cources/BigData/midterm/BX-Books.csv', encoding = 'unicode_escape',error_bad_lines=False,sep=";")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 9\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [4]:
print(raw.shape)
raw.head()

(1149780, 3)


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
raw_books.head()[["ISBN", "Book-Title"]]

,ISBN,Book-Title
0,0195153448,Classical Mythology
1,0002005018,Clara Callan
2,0060973129,Decision in Normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi


In [6]:
'''
Loải bỏ rating không hợp lệ, chỉ lấy user đã rate ít nhất 10 lần, chỉ lấy sách được rate bởi ít nhất 50 user
'''
train = raw.sample(n=1000000)

ratings_new=train

ratings_explicit = ratings_new[ratings_new['Book-Rating'] != 0]
ratings_implicit = ratings_new[ratings_new['Book-Rating'] == 0]


counts1 = ratings_explicit['User-ID'].value_counts()
counts2 = ratings_explicit['ISBN'].value_counts()
ratings_explicit = ratings_explicit[(ratings_explicit['User-ID'].isin(counts1[counts1 >= 20].index)) & (ratings_explicit['ISBN'].isin(counts2[counts2 >= 100].index))]
counts = ratings_explicit['Book-Rating'].value_counts()

n_users = ratings_explicit['User-ID'].unique().shape[0]
n_books= ratings_explicit['ISBN'].unique().shape[0]
print(ratings_explicit.shape)
print(n_users * n_books)
# ratings_matrix = ratings_explicit.pivot(index='ISBN', columns='User-ID', values='Book-Rating')
# ISBN = ratings_matrix.index
# userID = ratings_matrix.columns
## 5
# print(ratings_matrix.shape)

# ratings_matrix.fillna(0, inplace = True)
# ratings_matrix = ratings_matrix.astype(np.int32)
## 7

(7336, 3)
234304


In [7]:
'''
Tạo ma trận rating băng pyspark
'''
import pyspark.pandas as ps
ratings_matrix = ps.from_pandas(ratings_explicit)
ratings_matrix = ratings_matrix.pivot(index='User-ID', columns='ISBN', values='Book-Rating')
# ratings_matrix.dropna(axis='columns', how='all')
ratings_matrix.fillna(0, inplace = True)
ratings_matrix = ratings_matrix.astype(np.int32)
ratings_matrix.head()

ISBN,0060392452,0060502258,0060915544,0060928336,0060930535,0060934417,0060938455,0060959037,0060976845,0060987103,0061009059,006101351X,014028009X,0140293248,0142000205,0142001740,0156027321,0312195516,0312278586,0312291639,0312305060,0312924585,0316096199,0316284955,0316569321,0316601950,0316666343,0316769487,0316776963,0345313860,0345337662,0345339681,0345339703,0345342968,0345353145,0345361792,0345370775,0345417623,0345443284,0375700757,0375706771,0375707972,0375725784,0375726403,0375727345,0380789019,0380789035,0385265700,0385335482,0385484518,0385486804,0385492081,0385504209,0385505833,0385720106,0385722206,0439064864,0439064872,0439136350,0439136369,0439139597,043935806X,0440206154,044021145X,0440211727,0440213525,0440214041,0440220602,0440221471,044022165X,0440222656,0440224764,0440225701,0440226430,0440234743,0440236673,044023722X,0440241073,0446310786,0446364193,044651652X,0446605239,0446606812,0446608955,0446610038,0446672211,0449005615,0449212602,0452282152,0452282829,0553572997,0590353403,059035342X,0671003755,0671021001,0671027360,0671510053,0679746048,067976402X,0679781587,068484477X,0684872153,0684874350,0743237188,0743418174,0786868716,0804106304,0805063897,0842329129,0971880107,1400034779,1558743669
User-ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
37310,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
160434,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33862,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
62966,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0
74185,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
'''
Tạo ma trận rating không dùng spark
'''
ratings_matrix1 = ratings_explicit
ratings_matrix1 = ratings_matrix1.pivot(index='ISBN', columns='User-ID', values='Book-Rating')
# ratings_matrix.dropna(axis='columns', how='all')
ratings_matrix1.fillna(0, inplace = True)
ratings_matrix1 = ratings_matrix.astype(np.int32)
ratings_matrix1.shape

(2092, 112)

In [9]:
'''
Giảm chiều dữ liệu bằng spark PCA
'''
from pyspark.ml.feature import PCA
from pyspark.ml.feature import VectorAssembler
cols = ratings_matrix.columns.values
vectorAss = VectorAssembler(inputCols=cols, outputCol="features")
vdf = vectorAss.transform(ratings_matrix.to_spark(index_col="index"))
pca = PCA(k=50, inputCol="features", outputCol="out")
model = pca.fit(vdf)
rating_reduce = model.transform(vdf).select("out").rdd
rating_reduce = rating_reduce.zipWithIndex()

In [10]:
def user_based_book_recomender(n_recommend=10):
  n = 0
  # đưa ra gợi ý cho n người đầu tiên
  while n < n_recommend:
    # tính độ tương tự bằng khoảng cách cosine, lọc 10 người dùng tương tự nhất
    user = rating_reduce.filter(lambda x: x[1] == n).map(lambda x: x[0]).collect()
    distance = rating_reduce.map(lambda x: (x[1], cosine(list(x[0]), user)))
    index_top10 = distance.sortBy(lambda x: x[1]).map(lambda x: x[0]).take(10)
    recommend = []
    columns = ratings_matrix.columns.values
    top10 = ratings_matrix.iloc[index_top10]
    
    # Dự đoán rating, lọc ra 5 sách có rating dự đoán cao nhất để gợi ý cho người đọc
    for index, rating in enumerate(ratings_matrix.iloc[n].values):
      if rating == 0:
        recommend.append((columns[index], top10[columns[index]].mean()))  
    recommend.sort(reverse=True, key=lambda x: x[1])



    print(f"Độc giả với mã số {ratings_matrix.iloc[n].name} được giới thiệu những sách sau: \t")
    for i in recommend[0:5]:
        try:
            print("\t - " + str(raw_books.loc[raw_books['ISBN'] == i[0]]['Book-Title'].values[0]))
        except:
            print("Mã sách bị lỗi")
        
    print('\n-----------')
    n += 1

In [11]:
rating_reduce_nospark = rating_reduce.collect()
def user_based_book_recomender_no_spark(n_recommend=10):
  n = 0
  while n < n_recommend:

    user = rating_reduce_nospark[n][0]
    distance = list(map(lambda x: (cosine(x[0], user), x[1], x[0]), rating_reduce_nospark))
    distance.sort(key=lambda x: x[0])
    
    index_top10 = list(map(lambda x: x[1], distance[0:10]))
    recommend = []
    
    top10_nospark = ratings_matrix1.iloc[index_top10, :]
    top10_nospark_columns = ratings_matrix1.iloc[index_top10, :].columns.values
    for index, rating in enumerate(ratings_matrix1.iloc[n, :].values):
      if rating == 0:
        recommend.append((top10_nospark_columns[index], top10_nospark.loc[:, top10_nospark_columns[index]].mean())) 

    recommend.sort(reverse=True, key=lambda x: x[1])
    
    print(f"Độc giả với mã số {ratings_matrix1.iloc[n,:].name} được giới thiệu những sách sau: \t")
    for i in recommend[0:5]:
        try:
            print("\t - " + str(raw_books.loc[raw_books['ISBN'] == i[0]]['Book-Title'].values[0]))
        except:
            print("Mã sách bị lỗi")
    print('\n-----------')
    n += 1

In [12]:
start = time.time()
user_based_book_recomender_no_spark(5)
time.time() - start

Độc giả với mã số 37310 được giới thiệu những sách sau: 	
	 - While I Was Gone
	 - Into the Wild
	 - Summer Sisters
	 - The Chamber
	 - Stupid White Men ...and Other Sorry Excuses for the State of the Nation!

-----------
Độc giả với mã số 160434 được giới thiệu những sách sau: 	
	 - The Hobbit : The Enchanting Prelude to The Lord of the Rings
	 - Sphere
	 - Harry Potter and the Chamber of Secrets (Book 2)
	 - Harry Potter and the Prisoner of Azkaban (Book 3)
	 - Harry Potter and the Prisoner of Azkaban (Book 3)

-----------
Độc giả với mã số 33862 được giới thiệu những sách sau: 	
	 - We Were the Mulvaneys
	 - While I Was Gone
	 - A Map of the World
	 - A Heartbreaking Work of Staggering Genius
	 - Stupid White Men ...and Other Sorry Excuses for the State of the Nation!

-----------
Độc giả với mã số 62966 được giới thiệu những sách sau: 	
	 - The Nanny Diaries: A Novel
	 - While I Was Gone
	 - 1st to Die: A Novel
	 - The Girls' Guide to Hunting and Fishing
	 - Empire Falls

---------

271.63267946243286

In [13]:
start = time.time()
user_based_book_recomender(5)
time.time() - start

Độc giả với mã số 37310 được giới thiệu những sách sau: 	
	 - While I Was Gone
	 - Into the Wild
	 - Summer Sisters
	 - The Chamber
	 - Stupid White Men ...and Other Sorry Excuses for the State of the Nation!

-----------
Độc giả với mã số 160434 được giới thiệu những sách sau: 	
	 - The Hobbit : The Enchanting Prelude to The Lord of the Rings
	 - Sphere
	 - Harry Potter and the Chamber of Secrets (Book 2)
	 - Harry Potter and the Prisoner of Azkaban (Book 3)
	 - Harry Potter and the Prisoner of Azkaban (Book 3)

-----------
Độc giả với mã số 33862 được giới thiệu những sách sau: 	
	 - We Were the Mulvaneys
	 - While I Was Gone
	 - A Map of the World
	 - A Heartbreaking Work of Staggering Genius
	 - Stupid White Men ...and Other Sorry Excuses for the State of the Nation!

-----------
Độc giả với mã số 62966 được giới thiệu những sách sau: 	
	 - The Nanny Diaries: A Novel
	 - While I Was Gone
	 - 1st to Die: A Novel
	 - The Girls' Guide to Hunting and Fishing
	 - Empire Falls

---------

235.4693262577057

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# Tạo model, fix dữ liệu rating
ratings = sc.sparkContext.parallelize(ratings_explicit.values).filter(lambda x: x[1].isdigit())
ratingsRDD = ratings.map(lambda p: Row(userId=int(p[0]), bookID=int(p[1]), rating=float(p[2])))
ratings = sc.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="bookID", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)
# Dự đoán, đánh giá model
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


# Recommend
users = ratings.select(als.getUserCol()).distinct().limit(5)
userSubsetRecs = model.recommendForUserSubset(users, 10)

book = ratings.select(als.getItemCol()).distinct().limit(5)
bookSubSetRecs = model.recommendForItemSubset(book, 10)

Root-mean-square error = 9.099934619607877


In [15]:
for i in userSubsetRecs.collect():
  print(f"Đôc giả {i[0]} được giới thiệu")
  for j in i[1][0:5]:
    print("\t - Sách " + str(j[0]))
  print('\n-----------')


Đôc giả 33862 được giới thiệu
	 - Sách 60930535
	 - Sách 375725784
	 - Sách 312291639
	 - Sách 375700757
	 - Sách 440222656

-----------
Đôc giả 160434 được giới thiệu
	 - Sách 345339703
	 - Sách 439139597
	 - Sách 446608955
	 - Sách 375727345
	 - Sách 60987103

-----------
Đôc giả 74185 được giới thiệu
	 - Sách 449005615
	 - Sách 671510053
	 - Sách 60930535
	 - Sách 446608955
	 - Sách 684874350

-----------
Đôc giả 62966 được giới thiệu
	 - Sách 446608955
	 - Sách 61009059
	 - Sách 743418174
	 - Sách 60392452
	 - Sách 60938455

-----------
Đôc giả 133747 được giới thiệu
	 - Sách 345361792
	 - Sách 60934417
	 - Sách 316284955
	 - Sách 440241073
	 - Sách 375725784

-----------


In [16]:
for i in bookSubSetRecs.collect():
  print(f"Sách {i[0]} được giới thiệu cho")
  for j in i[1][0:5]:
    print("\t - Độc giả " + str(j[0]))
  print('\n-----------')

Sách 440225701 được giới thiệu cho
	 - Độc giả 110934
	 - Độc giả 136491
	 - Độc giả 224349
	 - Độc giả 268300
	 - Độc giả 22074

-----------
Sách 440221471 được giới thiệu cho
	 - Độc giả 110934
	 - Độc giả 135149
	 - Độc giả 11224
	 - Độc giả 30487
	 - Độc giả 196077

-----------
Sách 385335482 được giới thiệu cho
	 - Độc giả 8454
	 - Độc giả 115003
	 - Độc giả 30533
	 - Độc giả 261829
	 - Độc giả 123095

-----------
Sách 440241073 được giới thiệu cho
	 - Độc giả 110934
	 - Độc giả 259629
	 - Độc giả 56447
	 - Độc giả 232131
	 - Độc giả 196077

-----------
Sách 375726403 được giới thiệu cho
	 - Độc giả 110934
	 - Độc giả 232131
	 - Độc giả 136491
	 - Độc giả 144255
	 - Độc giả 68555

-----------
